<h3>For Regularly adding new data (features and labels) to the feature group. </h3>

In [7]:
from helper_functions import synthetic_data_model, patient_id_and_date
import hopsworks
from dotenv import load_dotenv
import os

In [8]:
model_dir = "synth_data_model"
model_path = os.path.join(model_dir, "synth_data_model.pkl")

model = synthetic_data_model(model_path)


In [9]:
new_synth_data = model.sample(100)
# new_synth_data.diabetes.value_counts()

In [10]:
new_synth_data = patient_id_and_date(new_synth_data)
# new_synth_data.head(3)

In [11]:
load_dotenv()
hopswork_key = os.getenv("HOPSWORK_KEY")

In [12]:
project = hopsworks.login(api_key_value=hopswork_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/112801
Connected. Call `.close()` to terminate connection gracefully.


In [13]:
fg = fs.get_or_create_feature_group(
    name="diabetes_features",
    version=1)

In [14]:
fg.insert(new_synth_data, write_options={"wait_for_job" : False})

2024-07-14 16:03:25,098 WARNING: FeatureGroupWarning: The ingested dataframe contains upper case letters in feature names: `['HbA1c_level']`. Feature names are sanitized to lower case in the feature store.



Uploading Dataframe: 0.00% |          | Rows 0/100 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: diabetes_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/112801/jobs/named/diabetes_features_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7ff544472860>, None)